## **Elaborazione di Immagini Mediche**
## Vessel Wall Segmentation Challenge - A.A. 2021/22 

### Lettura e salvataggio di immagini e maschere in formato tiff

Rigazio Sofia, Roccaro Lucia, Romano Anastasio, Ruzzante Elena


Collegamento a Google Drive e importazione delle librerie necessarie



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pydicom

import os
import pydicom
import glob
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from skimage.draw import polygon
from tifffile import imsave as imsave_tiff
from tqdm import tqdm
import json
from copy import deepcopy

Funzione per leggere i file DICOM ed i contour

In [ ]:
def readDicom(path):
  pi = os.path.basename(path).split('_')[1]
  dcm_size = len(glob.glob(path+'/*.dcm'))
  dcms = []
  dicom_slicei = 0
  for n_slicei in range(1,dcm_size+1):
    dicom_slicei = dicom_slicei+1
    path_dicom = path+'/E'+pi+'S101I%d.dcm'%dicom_slicei
    while os.path.exists(path_dicom) == False:
      dicom_slicei = dicom_slicei+1
      path_dicom = path+'/E'+pi+'S101I%d.dcm'%dicom_slicei
    dcms.append(path_dicom)

  dcm_f = pydicom.read_file(dcms[0]).pixel_array
  dcm_size = max(dcm_f.shape)

  cdcm1 = pydicom.read_file(dcms[0]).pixel_array

  cdcm_img = np.zeros((cdcm1.shape[0],cdcm1.shape[1],len(dcms)))
  dcm_img = np.zeros((dcm_size,dcm_size,len(dcms)))
  for dcmi in range(len(dcms)):
    cdcm = pydicom.read_file(dcms[dcmi]).pixel_array
    dcm_img[dcm_size//2-cdcm.shape[0]//2:dcm_size//2+cdcm.shape[0]//2,dcm_size//2-cdcm.shape[1]//2:dcm_size//2+cdcm.shape[1]//2,dcmi] = cdcm
    cdcm_img[:,:,dcmi] = cdcm

  # nomi delle immagini
  original_names = [dcms[i].split('/')[-1] for i in range(len(dcms))]
  original_names = [original_names[i].split('.')[0] for i in range(len(original_names))]

  return dcm_img, cdcm_img, original_names


def listContourSlices(qvsroot, dcm_img):
  avail_slices = []
  qvasimg = qvsroot.findall('QVAS_Image')
  for dicom_slicei in range(dcm_img.shape[2]):
    conts = qvasimg[dicom_slicei - 1].findall('QVAS_Contour')
    if len(conts):
      avail_slices.append(dicom_slicei)
  return avail_slices


def getContour(qvsroot,dicomslicei,conttype,dcmsz=720):
  qvasimg = qvsroot.findall('QVAS_Image')
  if dicomslicei - 1 > len(qvasimg):
    print('no slice', dicomslicei)
    return
  assert int(qvasimg[dicomslicei - 1].get('ImageName').split('I')[-1]) == dicomslicei
  conts = qvasimg[dicomslicei - 1].findall('QVAS_Contour')
  tconti = -1
  for conti in range(len(conts)):
    if conts[conti].find('ContourType').text == conttype:
      tconti = conti
      break
  if tconti == -1:
    # print('no such contour', conttype)
    return
  pts = conts[tconti].find('Contour_Point').findall('Point')
  contours = []
  for pti in pts:
    contx = float(pti.get('x')) / 512 * dcmsz 
    conty = float(pti.get('y')) / 512 * dcmsz 
    #if current pt is different from last pt, add to contours
    if len(contours) == 0 or contours[-1][0] != contx or contours[-1][1] != conty:
      contours.append([contx, conty])
  return np.array(contours)

Definizione delle directory e creazione della cartella che conterrà le immagini e le maschere

In [ ]:
current_dir = '/content/drive/MyDrive/Colab Notebooks/EIM/VesselWallSegmentationChallenge'
new_dataset_name = '3D_dataset'
path = os.path.join(current_dir,new_dataset_name)

try:
  os.mkdir(path)
except:
  pass

train_dir = os.path.join(current_dir,'DATASET/TRAIN')
validation_dir = os.path.join(current_dir,'DATASET/VALIDATION')
test_dir = os.path.join(current_dir,'DATASET/TEST')

# Creazione della cartella che conterrà file utili da passare tra gli script
JSON_path = os.path.join(current_dir,'PICKLE_and_JSON')
try:
  os.mkdir(JSON_path)
except:
  pass

Plot dei contour sovrapposti alle immagini per un soggetto e analisi per colonne

In [ ]:
casei = '0_P41_U'
pi = casei.split('_')[1]
dcm_img, cdcm_img, original_names = readDicom(train_dir+'/'+casei)

for arti in ['ICAL','ICAR','ECAL','ECAR']:
  cas_dir = train_dir + '/' + casei + '/CASCADE-' + arti
  qvs_path = cas_dir + '/E' + pi + 'S101_L.QVS'
  if os.path.exists(qvs_path) == True:
    qvsroot = ET.parse(qvs_path).getroot()
    avail_slices = listContourSlices(qvsroot, dcm_img)
    print('case',pi,'art',arti,'avail_slices',avail_slices)

    if len(avail_slices): # se c'è almeno una slice con segmentazione manuale
      nRows_cdcm = cdcm_img.shape[0]
      nRows_dcm = dcm_img.shape[0]
      nCols_cdcm = cdcm_img.shape[1]
      nCols_dcm = dcm_img.shape[1]

      dicom_slicei = avail_slices[0]
      print('Displaying the contours for the first slice for',arti)
      
      # contour su immagine quadrata
      lumen_cont = getContour(qvsroot,dicom_slicei,'Lumen',nCols_dcm)
      wall_cont = getContour(qvsroot,dicom_slicei,'Outer Wall',nCols_dcm)

      # contour su immagine originale - lume
      lumen_cont_cdcm = np.zeros_like(lumen_cont)
      lumen_cont_cdcm[:,0] = lumen_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
      lumen_cont_cdcm[:,1] = lumen_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)
      # contour su immagine originale - parete
      wall_cont_cdcm = np.zeros_like(wall_cont)
      wall_cont_cdcm[:,0] = wall_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
      wall_cont_cdcm[:,1] = wall_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)

      # plot
      plt.figure(figsize=(20,20))
      plt.imshow(cdcm_img[:,:,dicom_slicei],cmap=plt.cm.gray)
      plt.plot(lumen_cont_cdcm[:,0],lumen_cont_cdcm[:,1],'ro',markersize=1)
      plt.plot(wall_cont_cdcm[:,0],wall_cont_cdcm[:,1],'bo',markersize=1)
      plt.show()
    else:
      print (f"{arti}: no slices")

# Analisi dell'immagine per colonne per identificare il collo
central_slice = cdcm_img[:,:,int(round(cdcm_img.shape[2]/2))] 
std_col = np.std(central_slice, axis=0)

# visualizzazione
fig = plt.figure(figsize=(20, 5))
plt.title("Deviazione standard per colonne di una slice")
plt.plot(std_col)
fig.show()

Funzione per centrare il collo del soggetto rispetto all'immagine

In [ ]:
def centering(image,wall_mask,lumen_mask,to_shift):
  
  # se per il soggetto è già stato calcolato uno shift, teniamo quello già 
  # calcolato in modo da shiftare tutte le cartelle allo stesso modo, altrimenti
  # calcoliamo di quanto shiftare
  if to_shift is None:
    # centriamo l'immagine rispetto alla slice centrale
    central_slice = image[int(round(image.shape[0]/2))]

    # calcoliamo la deviazione dtandard delle colonne
    std_col = np.std(central_slice, axis=0)
    soglia = 10
    # troviamo gli indici per cui il vettore std_col è maggiore della soglia
    index = np.where(std_col > soglia)
    # troviamo il primo indice per cui la std supera la soglia partendo da 
    # sinistra (il primo del vettore index) -> corrisponde all'inizio del collo
    index_sx = index[0][0]
    # troviamo il primo indice per cui la std supera la soglia partendo da 
    # destra (l'ultimo del vettore index) -> corrisponde alla fine del collo
    index_dx = index[0][-1]

    # calcoliamo la dimensione dei due bordi
    bordo_dx = central_slice.shape[1] - index_sx
    bordo_sx = index_dx

    # centriamo l'immagine (se il numero è positivo, np.roll shifta da destra 
    # verso sinistra, se il numero è negativo, shifta nella direzione opposta)
    to_shift = int(round((bordo_dx-bordo_sx)/2))
  
  # effettuiamo lo shift dell'immagine
  image = np.roll(image, to_shift, axis=2)

  # per il Training e Validation set, shiftiamo anche le maschere
  if wall_mask is not None:
    wall_mask = np.roll(wall_mask, to_shift, axis=2)
    lumen_mask = np.roll(lumen_mask, to_shift, axis=2)

  return image, wall_mask, lumen_mask, to_shift

Funzione per la correzione di errori nel dataset

In [ ]:
def cleaning(X,Y_lumen,Y_wall,casei,arti,subject,set_):
  
  arti_new = None
  # analizziamo singolarmente ogni soggetto in quanto gli errori si presentano 
  # per ognuno in modo diverso. Se il soggetto passato alla function non rientra
  # tra quelli con errori, la funzione restituisce le martici invariale
  
  if casei == '0_P15_U':
    # per il P15, tutte le slice prima della EP15S101I363, segmentate come ICAR
    # devono essere ECAR
    if arti == 'ICAR':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAR'
      # troviamo l'indice della slice EP15S101I363
      idx = subject[arti]['name'].index('EP15S101I363')      
      # creiamo le matrici ECAR
      X_new = X[0:idx+1]
      Y_wall_new = Y_wall[0:idx+1]
      Y_lumen_new = Y_lumen[0:idx+1]     
      # rimuoviamo le slice ECAR dalle matrici ICAR
      X = X[idx+1:]
      Y_wall = Y_wall[idx+1:]
      Y_lumen = Y_lumen[idx+1:]
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      subject[arti_new]['name'] = subject[arti]['name'][0:idx+1]
      subject[arti_new]['number'] = subject[arti]['number'][0:idx+1]
      subject[arti_new+'_original']['name'] = subject[arti+'_original']['name'][0:idx+1]
      subject[arti_new+'_original']['number'] = subject[arti+'_original']['number'][0:idx+1]
      del subject[arti]['name'][0:idx+1]
      del subject[arti]['number'][0:idx+1]
      del subject[arti+'_original']['name'][0:idx+1]
      del subject[arti+'_original']['number'][0:idx+1]

  if casei == '0_P33_U':
    # per il P33, le slice EP33S101I331-332-336-337 sono rumore bianco con
    # segmentazione -> le eliminiamo
    to_delete = ['EP33S101I331','EP33S101I332','EP33S101I336','EP33S101I337']
    # troviamo l'indice delle slice e lo concateniamo all'array: facciamo try 
    # perché non tutte e quattro le slice sono presenti in tutti gli arti
    idx = []
    for slice_ in to_delete:
      try:
        idx.append(subject[arti]['name'].index(slice_))
      except:
        pass
    # rimuoviamo le slice dalle matrici
    X = np.delete(X,idx,axis=0)
    Y_wall = np.delete(Y_wall,idx,axis=0)
    Y_lumen = np.delete(Y_lumen,idx,axis=0)
    # sistemiamo il dizionario
    for slice_ in to_delete:
      try:
        idx = subject[arti]['name'].index(slice_)
        del subject[arti]['name'][idx]
        del subject[arti]['number'][idx]
        del subject[arti+'_original']['name'][idx]
        del subject[arti+'_original']['number'][idx]
      except:
        pass

  if casei == '0_P42_U':
    # per il P42, la slice EP42S101I318 è ICAR e dovrebbe essere ECAR, la slice
    # EP42S101I308 è ICAL e dovrebbe essere ECAL
    if arti == 'ICAR':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAR'
      # troviamo l'indice della slice EP42S101I318
      idx = subject[arti]['name'].index('EP42S101I318')      
      # creiamo le matrici ECAR
      X_new = X[idx]
      Y_wall_new = Y_wall[idx]
      Y_lumen_new = Y_lumen[idx]     
      # rimuoviamo la slice ECAR dalle matrici ICAR
      X = np.delete(X,idx,axis=0)
      Y_wall = np.delete(Y_wall,idx,axis=0)
      Y_lumen = np.delete(Y_lumen,idx,axis=0)
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      subject[arti_new]['name'] = [deepcopy(subject[arti]['name'][idx])]
      subject[arti_new]['number'] = [deepcopy(subject[arti]['number'][idx])]
      subject[arti_new+'_original']['name'] = [deepcopy(subject[arti+'_original']['name'][idx])]
      subject[arti_new+'_original']['number'] = [deepcopy(subject[arti+'_original']['number'][idx])]   
      del subject[arti]['name'][idx]
      del subject[arti]['number'][idx]
      del subject[arti+'_original']['name'][idx]
      del subject[arti+'_original']['number'][idx]
    if arti == 'ICAL':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAL'
      # troviamo l'indice della slice EP42S101I308
      idx = subject[arti]['name'].index('EP42S101I308')      
      # creiamo le matrici ECAL
      X_new = X[idx]
      Y_wall_new = Y_wall[idx]
      Y_lumen_new = Y_lumen[idx]     
      # rimuoviamo la slice ECAL dalle matrici ICAL
      X = np.delete(X,idx,axis=0)
      Y_wall = np.delete(Y_wall,idx,axis=0)
      Y_lumen = np.delete(Y_lumen,idx,axis=0)
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      subject[arti_new]['name'] = [deepcopy(subject[arti]['name'][idx])]
      subject[arti_new]['number'] = [deepcopy(subject[arti]['number'][idx])]
      subject[arti_new+'_original']['name'] = [deepcopy(subject[arti+'_original']['name'][idx])]
      subject[arti_new+'_original']['number'] = [deepcopy(subject[arti+'_original']['number'][idx])]
      del subject[arti]['name'][idx]
      del subject[arti]['number'][idx]
      del subject[arti+'_original']['name'][idx]
      del subject[arti+'_original']['number'][idx]

  if casei == '0_P4_U':
    # per il P4, tutte le slice prima della EP4S101I235, segmentate come ECAL
    # devono essere ICAL
    if arti == 'ECAL':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ICAL'
      # troviamo l'indice della slice EP4S101I235
      idx = subject[arti]['name'].index('EP4S101I235')    
      # creiamo le matrici ICAL
      X_new = X[0:idx+1]
      Y_wall_new = Y_wall[0:idx+1]
      Y_lumen_new = Y_lumen[0:idx+1]     
      # rimuoviamo le slice ICAL dalle matrici ECAL
      X = X[idx+1:]
      Y_wall = Y_wall[idx+1:]
      Y_lumen = Y_lumen[idx+1:]
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      subject[arti_new]['name'] = subject[arti]['name'][0:idx+1]
      subject[arti_new]['number'] = subject[arti]['number'][0:idx+1]
      subject[arti_new+'_original']['name'] = subject[arti+'_original']['name'][0:idx+1]
      subject[arti_new+'_original']['number'] = subject[arti+'_original']['number'][0:idx+1]
      del subject[arti]['name'][0:idx+1]
      del subject[arti]['number'][0:idx+1]
      del subject[arti+'_original']['name'][0:idx+1]
      del subject[arti+'_original']['number'][0:idx+1]

  if casei == '0_P27_U':
    # per il P27, la slice EP27S101I259 è ICAR e dovrebbe essere ECAR
    if arti == 'ICAR':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAR'
      # troviamo l'indice della slice EP27S101I259
      idx = subject[arti]['name'].index('EP27S101I259')      
      # creiamo le matrici ECAR
      X_new = X[idx]
      Y_wall_new = Y_wall[idx]
      Y_lumen_new = Y_lumen[idx]     
      # rimuoviamo la slice ECAR dalle matrici ICAR
      X = np.delete(X,idx,axis=0)
      Y_wall = np.delete(Y_wall,idx,axis=0)
      Y_lumen = np.delete(Y_lumen,idx,axis=0)
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      subject[arti_new]['name'] = [deepcopy(subject[arti]['name'][idx])]
      subject[arti_new]['number'] = [deepcopy(subject[arti]['number'][idx])]
      subject[arti_new+'_original']['name'] = [deepcopy(subject[arti+'_original']['name'][idx])]
      subject[arti_new+'_original']['number'] = [deepcopy(subject[arti+'_original']['number'][idx])]  
      del subject[arti]['name'][idx]
      del subject[arti]['number'][idx]
      del subject[arti+'_original']['name'][idx]
      del subject[arti+'_original']['number'][idx]

  if casei == '0_P36_U':
    # per il P36, la slice EP36S101I314 è ICAL e dovrebbe essere ECAL
    # le seguenti slice sono ICAR e dovrebbero errere ECAR: EP36S101I313, 
    # EP36S101I316, EP36S101I321, EP36S101I326, EP36S101I331, EP36S101I336, 
    # EP36S101I341, EP36S101I346, EP36S101I351, EP36S101I356
    if arti == 'ICAL':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAL'
      # troviamo l'indice della slice EP36S101I314
      idx = subject[arti]['name'].index('EP36S101I314')      
      # creiamo le matrici ECAL
      X_new = X[idx]
      Y_wall_new = Y_wall[idx]
      Y_lumen_new = Y_lumen[idx]     
      # rimuoviamo la slice ECAL dalle matrici ICAL
      X = np.delete(X,idx,axis=0)
      Y_wall = np.delete(Y_wall,idx,axis=0)
      Y_lumen = np.delete(Y_lumen,idx,axis=0)
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      subject[arti_new]['name'] = [deepcopy(subject[arti]['name'][idx])]
      subject[arti_new]['number'] = [deepcopy(subject[arti]['number'][idx])]
      subject[arti_new+'_original']['name'] = [deepcopy(subject[arti+'_original']['name'][idx])]
      subject[arti_new+'_original']['number'] = [deepcopy(subject[arti+'_original']['number'][idx])]
      del subject[arti]['name'][idx]
      del subject[arti]['number'][idx]
      del subject[arti+'_original']['name'][idx]
      del subject[arti+'_original']['number'][idx]
    if arti == 'ICAR':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAR'
      # slice da spostare
      to_move = ['EP36S101I313', 'EP36S101I316', 'EP36S101I321', 'EP36S101I326',
                 'EP36S101I331', 'EP36S101I336', 'EP36S101I341', 'EP36S101I346',
                 'EP36S101I351', 'EP36S101I356']
      # troviamo l'indice delle slice
      idx = []
      for slice_ in to_move:
        idx.append(subject[arti]['name'].index(slice_))
      # creiamo le matrici ECAR
      X_new = X[idx]
      Y_wall_new = Y_wall[idx]
      Y_lumen_new = Y_lumen[idx]     
      # rimuoviamo le slice ECAR dalle matrici ICAR
      X = np.delete(X,idx,axis=0)
      Y_wall = np.delete(Y_wall,idx,axis=0)
      Y_lumen = np.delete(Y_lumen,idx,axis=0)
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      for slice_ in to_move:
        idx = subject[arti]['name'].index(slice_)
        subject[arti_new]['name'].append(subject[arti]['name'][idx])
        subject[arti_new]['number'].append(subject[arti]['number'][idx])
        subject[arti_new+'_original']['name'].append(subject[arti+'_original']['name'][idx])
        subject[arti_new+'_original']['number'].append(subject[arti+'_original']['number'][idx]) 
        del subject[arti]['name'][idx]
        del subject[arti]['number'][idx]
        del subject[arti+'_original']['name'][idx]
        del subject[arti+'_original']['number'][idx]

  if casei == '0_P19_U':
    # per il P19 le seguenti slice sono ICAL e dovrebbero errere ECAL: 
    # EP19S101I330, EP19S101I332, EP19S101I337, EP19S101I342, EP19S101I347,
    # EP19S101I351, EP19S101I352, EP19S101I357, EP19S101I360, EP19S101I368
    if arti == 'ICAL':
      # cartella in cui devono salvate le matrici nuove
      arti_new = 'ECAL'
      # slice da spostare
      to_move = ['EP19S101I330', 'EP19S101I332', 'EP19S101I337', 'EP19S101I342',
                 'EP19S101I347', 'EP19S101I351', 'EP19S101I352', 'EP19S101I357',
                 'EP19S101I360', 'EP19S101I368']
      # troviamo l'indice delle slice
      idx = []
      for slice_ in to_move:
        idx.append(subject[arti]['name'].index(slice_))
      # creiamo le matrici ECAL
      X_new = X[idx]
      Y_wall_new = Y_wall[idx]
      Y_lumen_new = Y_lumen[idx]     
      # rimuoviamo le slice ECAL dalle matrici ICAL
      X = np.delete(X,idx,axis=0)
      Y_wall = np.delete(Y_wall,idx,axis=0)
      Y_lumen = np.delete(Y_lumen,idx,axis=0)
      # sistemiamo il dizionario
      subject[arti_new] = {'number': [], 'name': []}
      subject[arti_new+'_original'] = {'number': [], 'name': []}
      for slice_ in to_move:
        idx = subject[arti]['name'].index(slice_)
        subject[arti_new]['name'].append(subject[arti]['name'][idx])
        subject[arti_new]['number'].append(subject[arti]['number'][idx])
        subject[arti_new+'_original']['name'].append(subject[arti+'_original']['name'][idx])
        subject[arti_new+'_original']['number'].append(subject[arti+'_original']['number'][idx]) 
        del subject[arti]['name'][idx]
        del subject[arti]['number'][idx]
        del subject[arti+'_original']['name'][idx]
        del subject[arti+'_original']['number'][idx]

  # se c'erano dei problemi sul soggetto e alcune slice vanno spostate in 
  # un'altra cartella, creiamo la cartella per salvare le matrici nuove e le 
  # salviamo
  if arti_new is not None:
    path = os.path.join(current_dir,new_dataset_name,set_,casei,arti_new)
    try:
      os.mkdir(path)
    except:
      pass
    if len(X_new.shape) == 2: 
    # se c'è una sola slice, espandiamo le dimensioni da (x,x) a (1,x,x) in modo
    # da salvare le immagini come volumi e poterle successivamente trattare in
    # modo analogo alle altre immagini
      X_new = np.expand_dims(X_new,axis=0)
      Y_lumen_new = np.expand_dims(Y_lumen_new,axis=0)
      Y_wall_new = np.expand_dims(Y_wall_new,axis=0)
    # salvataggio delle immagini
    imsave_tiff(os.path.join(path,'image.tiff'),X_new)
    imsave_tiff(os.path.join(path,'lumen_mask.tiff'),Y_lumen_new)
    imsave_tiff(os.path.join(path,'wall_mask.tiff'),Y_wall_new)
  
  return X,Y_lumen,Y_wall,subject

Salvataggio dei volumi 3d e delle rispettive maschere in formato .tiff

In [ ]:
def salvataggio_tiff(dir,set_):

  # inizializzazione del dizionazio che conterrà le informazioni sulle immagini
  labels = {} 

  for v_subj, casei in tqdm(enumerate(os.listdir(dir)), total=len(os.listdir(dir))):
    
    # inizializzazione di una variabile che ci permette di shiftare tutte le 
    # immagini dello stesso soggetto dello stesso numero di colonne
    to_shift = None

    subject = {
      'n_slices': [], # numero totale di slice
      'original_shape' : [] # dimensioni originali delle slice
    }

    pi = casei.split('_')[1]
    dcm_img, cdcm_img, original_names = readDicom(os.path.join(dir,casei))
    subject['n_slices'] = cdcm_img.shape[2]
    subject['original_shape'] = cdcm_img.shape[0:2]

    slice_info = {
          'number': [], # numero della slice
          'name': [] # nome del file originale contenente la slice
        }

    # per il Test set salviamo tutte le slice
    if set_ == 'TEST':
      # aggiornamento del dizionario
      kept_slices = range(0,cdcm_img.shape[2]-1,10) # teniamo una slice su 10 per il Test set
      slice_info['number'] = [i for i in kept_slices]
      slice_info['name'] = [original_names[i] for i in kept_slices]
      subject['slices'] = slice_info 
      # salvataggio delle immagini
      X = np.zeros((len(kept_slices),cdcm_img.shape[0],cdcm_img.shape[1]), dtype=np.uint8)
      for i in range(len(kept_slices)):
        X[i] = cdcm_img[:,:,kept_slices[i]] # mettiamo il numero della slice come prima dimensione in X
      # centriamo il collo del soggetto rispetto all'immagine
      X,_,_,to_shift = centering(X,None,None,to_shift)
      # aggiorniamo il dizionario
      subject['shift'] = to_shift

    else: # TRAINING E VAL SET
      # inizializzazione a vuoto
      for key in ['ICAL_original','ICAR_original','ECAL_original','ECAR_original','ICAL','ICAR','ECAL','ECAR']:
        # XXXX_original contiene le slice disponibili prima del preprocessing
        # XXXX contiene le slice disponibili dopo la pulizia effettuata con il preprocessing
        subject[key] = slice_info

      for arti in ['ICAL','ICAR','ECAL','ECAR']:
        cas_dir = dir + '/' + casei + '/CASCADE-' + arti
        qvs_path = cas_dir + '/E' + pi + 'S101_L.QVS'

        # reinizializzazione a vuoto per ogni arti in modo che rimanga vuoto in
        # assenza di slice
        slice_info = {
          'number': [], # numero della slice
          'name': [] # nome del file originale contenente la slice
        }

        if os.path.exists(qvs_path) == True:
          qvsroot = ET.parse(qvs_path).getroot()
          avail_slices = listContourSlices(qvsroot, dcm_img)
          
          if len(avail_slices): # se c'è almeno una slice con segmentazione manuale
            # inseriamo le maschere nere
            if set_ == 'TRAIN':
              if arti == 'ICAL' or arti == 'ICAR':
                # tre maschere nere per le carotidi interne
                avail_slices.extend([5,35,70])
              else:
                # due maschere nere per le carotidi esterne
                avail_slices.extend([35,70])
            elif set_ == 'VALIDATION':
              # una maschera nera sia per interne che per esterne
              avail_slices.extend([70])

            nRows_cdcm = cdcm_img.shape[0]
            nRows_dcm = dcm_img.shape[0]
            nCols_cdcm = cdcm_img.shape[1]
            nCols_dcm = dcm_img.shape[1]

            X = np.zeros((len(avail_slices),nRows_cdcm,nCols_cdcm), dtype=np.uint8) # immagini
            Y_wall = np.zeros((len(avail_slices),nRows_cdcm,nCols_cdcm), dtype=np.float32) # maschere parete
            Y_lumen = np.zeros((len(avail_slices),nRows_cdcm,nCols_cdcm), dtype=np.float32) # maschere lume
            slice_names = [None]*len(avail_slices)

            index_to_remove = []
            for slice_counter, dicom_slicei in enumerate(avail_slices):
              # contour su immagine quadrata
              lumen_cont = getContour(qvsroot,dicom_slicei,'Lumen',nCols_dcm)
              wall_cont = getContour(qvsroot,dicom_slicei,'Outer Wall',nCols_dcm)

              # controllo se ci sono entrambi i contour
              if wall_cont is not None and lumen_cont is not None: 

                X[slice_counter] = cdcm_img[:,:,dicom_slicei]

                # contour su immagine originale - lume
                lumen_cont_cdcm = np.zeros_like(lumen_cont)
                lumen_cont_cdcm[:,0] = lumen_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
                riga = lumen_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)
                riga[riga > nRows_cdcm] = nRows_cdcm-1
                lumen_cont_cdcm[:,1] = riga

                # contour su immagine originale - parete
                wall_cont_cdcm = np.zeros_like(wall_cont)
                wall_cont_cdcm[:,0] = wall_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
                riga = wall_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)
                riga[riga > nRows_cdcm] = nRows_cdcm-1
                wall_cont_cdcm[:,1] = riga

                # creo le maschere a partire dai contour
                lumen_mask = np.zeros((nRows_cdcm,nCols_cdcm), dtype = np.float32)
                r = lumen_cont_cdcm[:,1]
                c = lumen_cont_cdcm[:,0]
                rr, cc = polygon(r, c)
                lumen_mask[rr, cc] = 1.0
                Y_lumen[slice_counter] = lumen_mask

                wall_mask = np.zeros((nRows_cdcm,nCols_cdcm), dtype = np.float32)
                r = wall_cont_cdcm[:,1]
                c = wall_cont_cdcm[:,0]
                rr, cc = polygon(r, c)
                wall_mask[rr, cc] = 1.0
                Y_wall[slice_counter] = wall_mask

              elif wall_cont is None and lumen_cont is None:
                X[slice_counter] = cdcm_img[:,:,dicom_slicei]

              else: # quando manca solo uno dei due contour -> la slice va eliminata
                index_to_remove.append(slice_counter)

            # rimozione delle slice per cui manca uno dei due contour
            X = np.delete(X,index_to_remove,axis=0)
            [avail_slices.pop(i) for i in index_to_remove]

            # salvataggio delle immagini e delle maschere come volumi 3d in immagini .tiff
            # creazione cartelle 
            path = os.path.join(current_dir,new_dataset_name,set_)
            try:
              os.mkdir(path)
            except:
              pass
            path = os.path.join(path,casei)
            try:
              os.mkdir(path)
            except:
              pass
            path = os.path.join(path,arti)
            try:
              os.mkdir(path)
            except:
              pass

            # centriamo il collo del soggetto rispetto all'immagine
            X,Y_wall,Y_lumen,to_shift = centering(X,Y_wall,Y_lumen,to_shift) 

            # aggiorniamo il dizionario -> salviamo le informazioni sia in 
            # XXXX_original sia in XXXX. quando rimuoveremo delle slice, faremo 
            # .pop() dalle liste XXXX e lasceremo inalterate le XXXX_original
            slice_info['number'] = deepcopy(avail_slices)
            slice_info['name'] = deepcopy([original_names[i] for i in avail_slices])
            
            subject[arti+'_original'] = deepcopy(slice_info)
            subject[arti] = deepcopy(slice_info)
            subject['shift'] = to_shift

            # richiamo della funzione per risolvere errori nelle segmentazioni 
            # manuali -> riorganizzazione delle slice errate nelle giuste 
            # cartelle
            X,Y_lumen,Y_wall,subject = cleaning(X,Y_lumen,Y_wall,casei,arti,subject,set_)

            # salvataggio delle immagini
            imsave_tiff(os.path.join(path,'image.tiff'),X)
            imsave_tiff(os.path.join(path,'lumen_mask.tiff'),Y_lumen)
            imsave_tiff(os.path.join(path,'wall_mask.tiff'),Y_wall)
    
    # aggiornamento del dizionario
    labels[casei] = subject

  return labels

In [ ]:
print('Saving images - Training Set')
labels_TRAIN = salvataggio_tiff(train_dir,'TRAIN')

print('\nSaving images - Validation Set')
labels_VAL = salvataggio_tiff(validation_dir,'VALIDATION')

print('\nSaving informations - Test Set')
labels_TEST = salvataggio_tiff(test_dir,'TEST')

# salvataggio dei dizionari in .json
with open(os.path.join(JSON_path,'labels_TRAIN.json'),"w") as f:
  json.dump(labels_TRAIN,f,indent = 4)
  f.close
with open(os.path.join(JSON_path,'labels_VAL.json'),"w") as f:
  json.dump(labels_VAL,f,indent = 4)
  f.close
with open(os.path.join(JSON_path,'labels_TEST.json'),"w") as f:
  json.dump(labels_TEST,f,indent = 4)
  f.close